In [ ]:
import torch
from math import sqrt
from torch import nn

In [ ]:
class Self_Attention_Muti_Head(nn.Module):
    def __init__(self, input_dim, k_dim, v_dim, nums_head):
        super(Self_Attention_Muti_Head, self).__init__()
        assert k_dim % nums_head == 0
        assert v_dim % nums_head == 0
        self.q = nn.Linear(input_dim, k_dim)
        self.k = nn.Linear(input_dim, k_dim)
        self.v = nn.Linear(input_dim, v_dim)

        self.nums_head = nums_head
        self.k_dim = k_dim
        self.v_dim = v_dim
        self._norm_fact = 1 / sqrt(k_dim // nums_head)

    def forward(self, x):
        batch_size = x.size(0)
      
        Q = self.q(x).view(batch_size, -1, self.nums_head, self.k_dim // self.nums_head).transpose(1, 2)  #[batch_size, nums_head, seq_len, k_dim // nums_head]
        K = self.k(x).view(batch_size, -1, self.nums_head, self.k_dim // self.nums_head).transpose(1, 2)
        V = self.v(x).view(batch_size, -1, self.nums_head, self.v_dim // self.nums_head).transpose(1, 2)
        print(x.shape)
        print(Q.size())

        atten = torch.matmul(Q, K.transpose(-1, -2)) * self._norm_fact
        atten = nn.Softmax(dim=-1)(atten)

        output = torch.matmul(atten, V).transpose(1, 2).contiguous().view(batch_size, -1, self.v_dim)
        return output
    

